In [11]:
import ipaddress
import random
import nmap
import masscan
import os

top_20_ports = ['80','23','443','21','22','25','3389','110','445','139','143','53','135','3306','8080','1723','111','995','993','5900']
ranges = ['192.168.3.0/24'] #,'10.129.90.0/24','10.10.10.0/24','192.168.2.0/24','172.16.0.0/12']
tracking = {}
for range in ranges:
    tracking[range]={'uphost_count':0,'responsive_ips':[],'responsive_ports':[]} # scan types needs to be updated to successfulscantype

class IPer():
    def __init__(self, ip_range):
        self.ip_range = ipaddress.IPv4Network('192.168.2.0/24')
    
    def get_samples(netrange,percentage):
        
        hosts = list(ipaddress.ip_network(netrange).hosts())
        print('TOTAL Possible IPs: {}'.format(len(hosts)))
        ten_percent = len(hosts)*percentage
        choosen1 = random.sample(hosts,int(ten_percent))
        after_choosen1 = list(set(hosts)-set(choosen1))
        choosen2 = random.sample(after_choosen1,int(ten_percent))

        after_choosen2 = list(set(after_choosen1)-set(choosen2))
        choosen3 = random.sample(after_choosen2,(int(ten_percent)))
        
        choosen1_strings = []
        for ip in choosen1:
            choosen1_strings.append(str(ip))
        
        choosen2_strings = []
        for ip in choosen2:
            choosen2_strings.append(str(ip))
            
        choosen3_strings = []
        for ip in choosen3:
            choosen3_strings.append(str(ip))
                                
        return choosen1_strings,choosen2_strings,choosen3_strings
    
    def get_my_local_ip():
        os.system("ip -br address | grep eth0  | cut -d 'P' -f 2 | cut -d ' ' -f 14 | cut -d '/' -f 1 > file.txt")
        with open("file.txt",'r') as f:
            ip = f.read()
            ip_strip = ip.strip()
            return ip_strip

    
def nmap_top_hundred(the_range):
    sample1,sample2,sample3 = IPer.get_samples(the_range,.2)
    scan1=''
    scan2=''
    scan3=''
    nm = nmap.PortScanner()
    print('SCAN #1')
    scan_data =nm.scan(hosts=' '.join(sample1), arguments='-T5 -n --top-ports 100 --exclude {}'.format())
    print('Uphosts: ',scan_data['nmap']['scanstats']['uphosts'])
    print(scan_data)
    for ip,data in scan_data['scan'].items():
        if data['status']['state'] == 'up':
            # determine which range the ip belongs to then add to uphost_count, and update responsive IPs,
            tracking[the_range]['uphost_count'] += 1
            tracking[the_range]['responsive_ips'].append([ip,'nmap_portscan'])
    return scan_data


##################
# TO DO:
#       1. Add checks to determine fastest rate for network allowed
def masscan_ports(the_range,top_20_ports):
        #will use samples
        sample1,sample2,sample3 = IPer.get_samples(the_range,.3)
        samples = [sample1,sample2,sample3]
        ms = masscan.PortScanner()
        
        for sample in samples:
            print(sample)
            if tracking[the_range] == 0:
                ms.scan(' '.join(sample),ports='{}'.format(','.join(top_20_ports)),arguments="--router-mac 66-55-44-33-22-11 -e eth0 --max-rate 30000")
                #place holder return value from historic run on hackthebox.com
                '''
                ms = {ipaddress.ip_address(('10.129.90.245')): ['tcp', [80, 135, 139, 445]],
                      ipaddress.ip_address(('10.129.90.241')): ['tcp', [80]],
                      ipaddress.ip_address(('10.129.90.146')): ['tcp', [22, 80]],
                      ipaddress.ip_address(('10.129.90.220')): ['tcp', [80]],
                      ipaddress.ip_address(('10.129.90.128')): ['tcp', [80]]}
                '''
                the_count = 0
                for ip,ports in ms.items():
                    print("THIS IS IP: {}/tTHIS IS PORTS: {}".format(ip,ports))
                    if len(ports) > 0:
                        the_count += 1
                        tracking[the_range]['uphost_count'] += 1
                        tracking[the_range]['responsive_ips'].append(ip)
                        tracking[the_range]['responsive_ports'].append([ports,'masscan_portscan'])
                
def fping_sweep(the_range):
    print("[*]RUNNING PING SWEEP[*]")
    cmd='fping -4 --addr -r 1 -a -i 1 -g {} 2>/dev/null >> fping_uphosts.txt'.format(the_range)
    out = os.system(cmd)
    with open('fping_uphosts.txt','r') as f:
        lines = f.readlines()
        if len(lines) > 0:
            scanner_ip = IPer.get_my_local_ip()
            for line in lines:
                if scanner_ip not in line:
                    tracking[the_range]['uphost_count'] += 1
                    tracking[the_range]['responsive_ips'].append([line.strip(),'fping_ping'])
                else:
                    pass
    os.system('rm fping_uphosts.txt')
    print('[*]PING SWEEP COMPLETE[*]')
    
    
def unicorn_ports(the_range,top_20_ports):
    sample1,sample2,sample3 = IPer.get_samples(the_range,.1)
    samples = [sample1,sample2,sample3]
    for sample in samples:
        if tracking[the_range]["uphost_count"] >0:
            pass
        else:
            # sudo unicornscan --immediate -mT -R 1 -r100000 -i eth0 192.168.3.0/24 --ports 80,443,22
            cmd='sudo unicornscan -mT -R 1 -r100000 -i eth0 {} --ports {} >> unicornscan.out'.format(' '.join(sample),','.join(top_20_ports))
            out = os.system(cmd)
            get_ports = "cat unicornscan.out | cut -d '[' -f2 | cut -d ']' -f1 | cut -d ' ' -f 4 >>ports.out"
            os.system(get_ports)
            get_ips = "cat unicornscan.out | cut -d 'm' -f2 | cut -d ' ' -f2 >> unicorn_ips.out"
            os.system(get_ips)

            ips = []
            ports = []
            with open('unicorn_ips.out','r') as f:
                d = f.readlines()
                for ip in d:
                    ips.append(ip.strip())
            with open('ports.out','r') as f:
                d = f.readlines()
                for port in d:
                    ports.append(port.strip())
            ip_port = list(zip(ips,ports))
            for each in ip_port:
                if each[0] not in tracking[the_range]['responsive_ips'] and each[0] !='':
                    tracking[the_range]['uphost_count'] += 1
                    tracking[the_range]['responsive_ips'].append([each[0],'unicorn_scan'])

    
        

        

In [12]:
for range in ranges:
    fping_sweep(range)

[*]RUNNING PING SWEEP[*]
[*]PING SWEEP COMPLETE[*]


In [13]:
tracking

{'192.168.3.0/24': {'uphost_count': 0,
  'responsive_ips': [],
  'responsive_ports': []}}

In [14]:
unicorn_ports(range,top_20_ports)

TOTAL Possible IPs: 254


In [15]:
tracking

{'192.168.3.0/24': {'uphost_count': 102,
  'responsive_ips': [['192.168.3.27', 'unicorn_scan'],
   ['192.168.3.33', 'unicorn_scan'],
   ['192.168.3.195', 'unicorn_scan'],
   ['192.168.3.27', 'unicorn_scan'],
   ['192.168.3.33', 'unicorn_scan'],
   ['192.168.3.195', 'unicorn_scan'],
   ['192.168.3.27', 'unicorn_scan'],
   ['192.168.3.33', 'unicorn_scan'],
   ['192.168.3.195', 'unicorn_scan'],
   ['192.168.3.31', 'unicorn_scan'],
   ['192.168.3.95', 'unicorn_scan'],
   ['192.168.3.195', 'unicorn_scan'],
   ['192.168.3.27', 'unicorn_scan'],
   ['192.168.3.33', 'unicorn_scan'],
   ['192.168.3.195', 'unicorn_scan'],
   ['192.168.3.31', 'unicorn_scan'],
   ['192.168.3.95', 'unicorn_scan'],
   ['192.168.3.195', 'unicorn_scan'],
   ['192.168.3.27', 'unicorn_scan'],
   ['192.168.3.33', 'unicorn_scan'],
   ['192.168.3.195', 'unicorn_scan'],
   ['192.168.3.31', 'unicorn_scan'],
   ['192.168.3.95', 'unicorn_scan'],
   ['192.168.3.195', 'unicorn_scan'],
   ['192.168.3.24', 'unicorn_scan'],
   ['192

In [ ]:
# nmap port scan top 100 ports if ping and masscan fail to get results        
for ip_range,values in tracking.items():
    if values['uphost_count'] <= 1:
        nmap_top_hundred(ip_range)
        
        

In [84]:
IPer.get_my_local_ip()


10.10.10.8


'10.10.10.8'

In [55]:
#run aping s1weep stop when you get a hit
'''                    
for range in ranges:
    scan1 = ping_sweep_nmap(range)
    if tracki

ng[range]['uphost_count'] == 0:
        #run second ping sweep
        scan2 = ping_sweep_nmap(range)
        if tracking[range]['uphost_count'] == 0:
            #run third ping sweep
            scan3 = ping_sweep_nmap(range)
            if tracking[range]['uphost_count'] == 0:
                # Start port scanning stop if you get a hit
                masscan_ports(range,top_20_ports)
                if tracking[range]['uphost_count'] == 0:
                    masscan_ports(range,top_20_ports)
                    if tracking[range]['uphost_count'] == 0:
                        masscan_ports(range,top_20_ports)
                    
''' 

"                    \nfor range in ranges:\n    scan1 = ping_sweep_nmap(range)\n    if tracki\n\nng[range]['uphost_count'] == 0:\n        #run second ping sweep\n        scan2 = ping_sweep_nmap(range)\n        if tracking[range]['uphost_count'] == 0:\n            #run third ping sweep\n            scan3 = ping_sweep_nmap(range)\n            if tracking[range]['uphost_count'] == 0:\n                # Start port scanning stop if you get a hit\n                masscan_ports(range,top_20_ports)\n                if tracking[range]['uphost_count'] == 0:\n                    masscan_ports(range,top_20_ports)\n                    if tracking[range]['uphost_count'] == 0:\n                        masscan_ports(range,top_20_ports)\n                    \n"